In [1]:
# import dependencies

#**IMdb Reviews classification using Tensorflow**

in this notebook im going to classify positive and negative reviews based on the comments, so the steps are...

- loading the datasets from the tensorflow datsets with samples ( train - 25000 , test - 25000) texts

- Data preprocessing and exploration

- tokenization ,sequences and padding the texts

- Training the model with LSTM( Bidirectional) layers

- Predicting the Sample texts


In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds

#**1. Loading the dataset**

loading the datasets from the tfds( tensorflow datasets ) import tensorflow_datasets as tfds

In [3]:
# loading the datasets
# with_info - the information of the dataset

data , info = tfds.load("imdb_reviews/plain_text", with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAZE9R6/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAZE9R6/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAZE9R6/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


splitting the datasets for training and testing 

In [9]:
train_data, test_data = data["train"], data["test"]

checking the length of the datasets of both train and test (25000)

In [10]:

len(list(train_data)) , len(list(test_data))

(25000, 25000)

#**2. Data preprocessing and Exploration**

In [11]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

from the above info , that data's from the datasets are 1. texts and 2. labels (similar to json format)

okay, now will see some of the data

In [61]:
# lets see the data text reviews

for reviews in train_data.take(5):
  print(reviews[0].numpy().decode("utf-8"))
  print(reviews[1].numpy())
  print("\n")



This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
0


I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was 

we notice that labels in 0 and 1 format, moreover

       0 ----> negative reviews
       1-----> positive reviews

Doing the same steps and store the text and labels in a list after thet convert to a numpy array for same train and test data 

In [13]:
# now do that for train and test

def preprocess(data):
  sentences = []
  labels = []

  for i in data:
  
    sentences.append(i[0].numpy().decode("utf-8"))
    labels.append(i[1].numpy())

  return sentences, labels


training_text, training_label = np.array(preprocess(train_data))
testing_text, testing_label = np.array(preprocess(test_data))
training_label = training_label.astype("int")
testing_label = testing_label.astype("int")


In [14]:
training_label

array([0, 0, 0, ..., 0, 0, 1])

In [15]:

training_text.shape , training_label.shape, testing_text.shape, testing_label.shape

((25000,), (25000,), (25000,), (25000,))

we need to find the maximum length of the training texts needed for padding 

In [62]:
max_len_sequences = max([len(i) for i in training_text])

max_len_sequences

13704

In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#**3. Tokenizing, Sequence and Padding**

Tokenize the training text data

In [63]:

# tokenization
vocab_size = 68379
tokenize = Tokenizer(num_words= vocab_size , oov_token= "<oov>")
tokenize.fit_on_texts(training_text)
word_index = tokenize.word_index
word_index, len(word_index)

({'<oov>': 1,
  'the': 2,
  'and': 3,
  'a': 4,
  'of': 5,
  'to': 6,
  'is': 7,
  'br': 8,
  'in': 9,
  'it': 10,
  'i': 11,
  'this': 12,
  'that': 13,
  'was': 14,
  'as': 15,
  'for': 16,
  'with': 17,
  'movie': 18,
  'but': 19,
  'film': 20,
  'on': 21,
  'not': 22,
  'you': 23,
  'are': 24,
  'his': 25,
  'have': 26,
  'he': 27,
  'be': 28,
  'one': 29,
  'all': 30,
  'at': 31,
  'by': 32,
  'an': 33,
  'they': 34,
  'who': 35,
  'so': 36,
  'from': 37,
  'like': 38,
  'her': 39,
  'or': 40,
  'just': 41,
  'about': 42,
  "it's": 43,
  'out': 44,
  'if': 45,
  'has': 46,
  'some': 47,
  'there': 48,
  'what': 49,
  'good': 50,
  'more': 51,
  'when': 52,
  'very': 53,
  'up': 54,
  'no': 55,
  'time': 56,
  'she': 57,
  'even': 58,
  'my': 59,
  'would': 60,
  'which': 61,
  'only': 62,
  'story': 63,
  'really': 64,
  'see': 65,
  'their': 66,
  'had': 67,
  'can': 68,
  'were': 69,
  'me': 70,
  'well': 71,
  'than': 72,
  'we': 73,
  'much': 74,
  'been': 75,
  'bad': 76,
  '

sequence and padding with training text data

In [19]:

train_sequences = tokenize.texts_to_sequences(training_text)
train_padded = pad_sequences(train_sequences, maxlen = max_len_sequences, truncating= "post" ) 

In [20]:
train_padded.shape

(25000, 13704)

sequence and padding with testing text data

In [21]:
# for testing
test_sequence = tokenize.texts_to_sequences(testing_text)
test_padded = pad_sequences(test_sequence, maxlen= max_len_sequences)

In [22]:
test_padded.shape

(25000, 13704)

#**4. Training the model**

#**Training our model**

#**Define our model**

- we are using bidirectional LSTM for our model [link for the Documentation](https://keras.io/api/layers/recurrent_layers/lstm)


In [64]:
# model define

model = keras.Sequential([
      keras.layers.Embedding(vocab_size, 16, input_length= max_len_sequences),
      keras.layers.Bidirectional(keras.layers.LSTM(32)),
      keras.layers.Dense(64, activation= "relu"),
      keras.layers.Dense(1, activation= "sigmoid")                          
])

model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 13704, 16)         1094064   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 1,110,833
Trainable params: 1,110,833
Non-trainable params: 0
_________________________________________________________________


run the below code to start training

it will take 45 minutes to train so need to wait 

In [24]:
history = model.fit(train_padded, training_label, epochs = 15,batch_size = 128, validation_data=(test_padded, testing_label))

Epoch 1/15
196/196 [==============================] - 631s 3s/step - loss: 0.4523 - accuracy: 0.7688 - val_loss: 0.2866 - val_accuracy: 0.8829
Epoch 2/15
196/196 [==============================] - 627s 3s/step - loss: 0.1874 - accuracy: 0.9304 - val_loss: 0.2994 - val_accuracy: 0.8746
Epoch 3/15
196/196 [==============================] - 628s 3s/step - loss: 0.0989 - accuracy: 0.9670 - val_loss: 0.3417 - val_accuracy: 0.8636
Epoch 4/15
196/196 [==============================] - 628s 3s/step - loss: 0.0575 - accuracy: 0.9823 - val_loss: 0.5756 - val_accuracy: 0.8589
Epoch 5/15
196/196 [==============================] - 627s 3s/step - loss: 0.0415 - accuracy: 0.9866 - val_loss: 0.5511 - val_accuracy: 0.8566
Epoch 6/15
196/196 [==============================] - 627s 3s/step - loss: 0.0206 - accuracy: 0.9949 - val_loss: 0.5979 - val_accuracy: 0.8557
Epoch 7/15
196/196 [==============================] - 628s 3s/step - loss: 0.0099 - accuracy: 0.9976 - val_loss: 0.8114 - val_accuracy: 0.8498

After the training as completed save the model by running the below code

- if you need to download the model it will be available in the folder section

In [25]:
model.save("imdb_reviews.h5")

In [26]:
model_path = "/content/imdb_reviews.h5"

#**Loading the model**

- we're using the saved model for loading and make prediction

In [65]:
from keras.models import load_model

model1 = load_model(model_path)

In [66]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 13704, 16)         1094064   
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                12544     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,110,833
Trainable params: 1,110,833
Non-trainable params: 0
_________________________________________________________________


#**Prediction with sample text**

In [57]:
def prediction(model1,text):
  sequences = tokenize.texts_to_sequences([text])
  padded = pad_sequences(sequences, maxlen = max_len_sequences, truncating = "post")
  reviews = model1.predict(padded)

  if reviews[0]>0.5:
    print("it is a positive percent : ",str(reviews[0][0]*100)[:3])
  else:
    print("it is a negative percent : ",str(reviews[0][0]*100)[:3])
  
  




calling the function with the example texts

In [58]:
text = "when the man saved the women and she were alive because of him. i am not liking this movie very boring"
prediction(model1, text)

it is a negative percent :  0.4


In [59]:
text = "again batman saved the world"
prediction(model1, text)

it is a positive percent :  86.


In [60]:
text = "this movie is inappropiate for children it's kind of scary even for adults"
prediction(model1, text)

it is a negative percent :  5.2


#**well, it was a fun and thanks for watching this.....**

- trained model will be available in github 

- soon i can attach the notebook for loading the trained model without retraining